<a href="https://colab.research.google.com/github/arcossci/agentic/blob/master/adk_tutorial_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construye su primer equipo de agente inteligente: un bot de clima progresivo con ADK

Este tutorial se extiende desde [QuickStart Ejemplo] (https://google.github.io/adk-docs/get-started/quickstart/) para [agente kit de desarrollo] (https://google.github.io/adk-docs/get-started/).Ahora, estás listo para sumergirte más profundo y construir un sistema más sofisticado, ** múltiple agente **.

Nos embarcaremos en la construcción de un ** equipo de agente de bot **, colocando progresivamente las características avanzadas en una base simple.Comenzando con un solo agente que puede buscar el clima, agregaremos incrementalmente capacidades como:

* Aprovechando diferentes modelos AI (Gemini, GPT, Claude).
* Diseño de sub-agentes especializados para tareas distintas (como saludos y despedidas).
* Habilitando la delegación inteligente entre agentes.
* Darle a los agentes memoria utilizando el estado de sesión persistente.
* Implementación de barandillas de seguridad cruciales utilizando devoluciones de llamada.

** ¿Por qué un equipo de botes meteorológicos? **

Este caso de uso, aunque aparentemente simple, proporciona un lienzo práctico y identificable para explorar conceptos de ADK básicos esenciales para construir aplicaciones de agente complejas del mundo real.Aprenderá a estructurar las interacciones, administrar el estado, garantizar la seguridad y orquestar múltiples "cerebros" de IA trabajando juntos.

** ¿Qué es ADK de nuevo? **

Como recordatorio, ADK es un marco de Python diseñado para optimizar el desarrollo de aplicaciones alimentadas por modelos de idiomas grandes (LLM).Ofrece bloques de construcción robustos para crear agentes que puedan razonar, planificar, utilizar herramientas, interactuar dinámicamente con los usuarios y colaborar de manera efectiva dentro de un equipo.

** En este tutorial avanzado, dominará: **

*✅ ** Definición y uso de herramientas: ** Crafting Python Functions ('Tools`) que otorgan a los agentes habilidades específicas (como obtener datos) e instruir a los agentes sobre cómo usarlas de manera efectiva.
*✅ ** Flexibilidad multi-LLM: ** Configuración de agentes para utilizar varios LLM principales (Gemini, GPT-4O, Claude Sonnet) a través de la integración de litellm, lo que le permite elegir el mejor modelo para cada tarea.
*✅ ** Delegación y colaboración del agente: ** Diseño de subcásgentes especializados y habilita el enrutamiento automático (`flujo automático ') de las solicitudes de los usuarios al agente más apropiado dentro de un equipo.
*✅ ** Estado de sesión para la memoria: ** Utilizando el `estado de sesión` y` herramientas de toolcontext` para permitir que los agentes recuerden la información en los giros conversacionales, lo que lleva a más interacciones contextuales.
*✅ ** Las barandillas de seguridad con devoluciones de llamada: ** Implementación de `antes_model_callback` y` antes_tool_callback` para inspeccionar, modificar o bloquear las solicitudes/uso de la herramienta basado en reglas predefinidas, mejorar la seguridad y el control de la aplicación.

** Expectativa estatal final: **

Al completar este tutorial, habrá creado un sistema funcional de botes meteorológicos de varios agentes.Este sistema no solo proporcionará información meteorológica, sino que también manejará las sutilezas de conversación, recuerde la última ciudad verificada y operará dentro de los límites de seguridad definidos, todos orquestados usando ADK.

** Requisitos previos: **

*✅ ** Comprensión sólida de la programación de Python. **
*✅ ** Familiaridad con modelos de idiomas grandes (LLM), API y el concepto de agentes. **
*❗ ** Crucialmente: finalización de los tutoriales ADK QuickStart o el conocimiento fundamental equivalente de los conceptos básicos de ADK (agente, corredor, sessionservice, uso básico de herramientas). ** Este tutorial se basa directamente en esos conceptos.
*✅ ** API teclas ** Para los LLM que tiene la intención de usar (por ejemplo, Google AI Studio para Gemini, plataforma OpenAI, consola antrópica).

** ¿Listo para construir su equipo de agentes?Vamos a sumergirnos! **

In [1]:
# Title Paso 0: Configuración e instalación
# Instale ADK y Litellm para soporte de múltiples modelos

!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.3 MB/s eta 0:00:00
Installation complete.


In [3]:
# Title Importar bibliotecas necesarias
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
from google.colab import userdata

import warnings
# Ignorar todas las advertencias
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [4]:
# Title Configure las teclas API (¡reemplace con sus claves reales!)

# --- IMPORTANTE: Reemplace los marcadores de posición con sus claves API reales ---

# Gemini API Key (Obtenga el estudio de Google AI: https://aistudio.google.com/app/apikey)
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')

# Clave de API de OpenAI (obtenga de la plataforma Operai: https://platform.openai.com/api- keys)
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Clave de API antrópica (obtenga de la consola antrópica: https://console.anthropic.com/settings/keys)
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')


# --- Verificar claves (verificación opcional) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configurar ADK para usar las teclas API directamente (no Vertex AI para esta configuración de múltiples modelos)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown


API Keys Set:
Google API Key set: Yes
OpenAI API Key set: Yes
Anthropic API Key set: Yes


In [16]:
# --- Definir constantes de modelo para facilitar el uso ---

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash-exp"

# Nota: Los nombres de modelos específicos pueden cambiar.Consulte la documentación de litellm/proveedor.
MODEL_GPT_4O = "openai/gpt-4o-mini"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-sonnet-20240229"


print("\nEnvironment configured.")


Environment configured.


---

## Paso 1: Su primer agente \- Búsqueda meteorológica básica

Comencemos construyendo el componente fundamental de nuestro bot: un solo agente capaz de realizar una tarea específica: buscar información meteorológica.Esto implica crear dos piezas principales:

1. ** Una herramienta: ** Una función de Python que equipa al agente con la*habilidad*para obtener datos meteorológicos.
2. ** Un agente: ** El AI "cerebro" que comprende la solicitud del usuario, sabe que tiene una herramienta meteorológica y decide cuándo y cómo usarla.

---

** 1 \.Defina la herramienta (`get_weather`) **

En ADK, ** Herramientas ** son los bloques de construcción que brindan a los agentes capacidades concretas más allá de la generación de texto.Típicamente son funciones de Python regulares que realizan acciones específicas, como llamar a una API, consultar una base de datos o realizar cálculos.

Nuestra primera herramienta proporcionará un informe meteorológico * simulado *.Esto nos permite centrarnos en la estructura del agente sin necesidad de claves API externas todavía.Más tarde, podría intercambiar fácilmente esta función simulada con una que llame a un servicio meteorológico real.

** Concepto clave: los documentos son cruciales \! ** El LLM del agente depende en gran medida del ** documento ** ** para comprender:

* * Qué * hace la herramienta.
* * Cuando * usarlo.
* * Qué argumentos * requiere (`ciudad: str`).
* * Qué información * devuelve.

** Las mejores prácticas: ** Escribe documentos claros, descriptivos y precisos para tus herramientas.Esto es esencial para que el LLM use la herramienta correctamente.

In [17]:
# Title Defina la herramienta get_weather
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    print(f"--- Tool: get_weather called for city: {city} ---") # Log tool execution
    city_normalized = city.lower().replace(" ", "") # Basic normalization

# # Datos meteorológicos simulados
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

# Ejemplo de uso de la herramienta (prueba opcional)
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Paris ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Paris'."}


---

** 2 \.Defina el agente (`Weather_agent`) **

Ahora, creemos el ** agente ** mismo.Un `agente` en ADK orquesta la interacción entre el usuario, el LLM y las herramientas disponibles.

Lo configuramos con varios parámetros clave:

* `Nombre`: un identificador único para este agente (por ejemplo," clima \ _agent \ _v1 ").
* `Model`: Especifica qué LLM usar (por ejemplo,` model_gemini_2_5_pro`).Comenzaremos con un modelo de Géminis específico.
* `Descripción`: un resumen conciso del propósito general del agente.Esto se vuelve crucial más adelante cuando otros agentes deben decidir si delegar tareas a * este * agente.
* `Instrucción`: orientación detallada para el LLM sobre cómo comportarse, su personalidad, sus objetivos y específicamente * cómo y cuándo * utilizar sus 'herramientas' asignadas.
* `Herramientas`: una lista que contiene las funciones reales de la herramienta Python que el agente puede usar (por ejemplo,` [get_weather] `).

** Las mejores prácticas: ** Proporcione indicaciones claras y específicas de `instrucciones`.Cuanto más detalladas son las instrucciones, mejor será la LLM para comprender su papel y cómo usar sus herramientas de manera efectiva.Sea explícito sobre el manejo de errores si es necesario.

** Las mejores prácticas: ** elige valores descriptivos de `nombre` y` description`.Estos se usan internamente por ADK y son vitales para características como la delegación automática (cubierto más adelante).

In [18]:
# Title Defina el agente meteorológico
# Use una de las constantes modelo definidas anteriormente
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Starting with Gemini

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'weather_agent_v1' created using model 'gemini-2.0-flash-exp'.


---

** 3 \.Configurar corredor y servicio de sesión **

Para administrar conversaciones y ejecutar el agente, necesitamos dos componentes más:

* `SessionService`: responsable de administrar el historial de conversación y el estado para diferentes usuarios y sesiones.El 'InMemorySessionService' es una implementación simple que almacena todo en la memoria, adecuada para pruebas y aplicaciones simples.Realiza un seguimiento de los mensajes intercambiados.Exploraremos más la persistencia del estado en el paso 4 \.
* `Runner`: el motor que orquesta el flujo de interacción.Toma la entrada del usuario, la enruta al agente apropiado, gestiona las llamadas a la LLM y las herramientas basadas en la lógica del agente, maneja las actualizaciones de la sesión a través del `sessionService` y produce eventos que representan el progreso de la interacción.

In [19]:
# Title Configuración de servicio de sesión y corredor

# --- Gestión de la sesión ---
# Concepto clave: SessionService Stores Historia y estado de conversación.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de interacción
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Cree la sesión específica donde sucederá la conversación
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Concepto clave: Runner orquesta el bucle de ejecución del agente.
runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'
Runner created for agent 'weather_agent_v1'.


---

** 4 \.Interactuar con el agente **

Necesitamos una forma de enviar mensajes a nuestro agente y recibir sus respuestas.Dado que las llamadas de LLM y las ejecuciones de herramientas pueden llevar tiempo, el 'Runner` de ADK funciona de manera asincrónica.

Definiremos una función de `` async` aelper (`call_agent_async`) que:

1. Toma una cadena de consulta de usuario.
2. Lo empaqueta en el formato ADK `Content`.
3. Llama a `Runner.run_async`, proporcionando el contexto del usuario/sesión y el nuevo mensaje.
4. Itera a través de los ** eventos ** cedidos por el corredor.Los eventos representan pasos en la ejecución del agente (por ejemplo, llamada de herramienta solicitada, resultado de la herramienta recibido, pensamiento intermedio de LLM, respuesta final).
5. Identifica e imprime la respuesta final ** ** Evento usando `event.is_final_esponse ()`.

** ¿Por qué `async`? ** Las interacciones con LLM y potencialmente herramientas (como las API externos) son operaciones de E/S.El uso de `Asyncio` permite que el programa maneje estas operaciones de manera eficiente sin bloquear la ejecución.

In [21]:
# Title Definir la función de interacción del agente

from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

# # Prepare el mensaje del usuario en formato ADK
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

# # Concepto de clave: Run_async ejecuta la lógica del agente y produce eventos.
# # Nos iteramos a través de eventos para encontrar la respuesta final.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
# # Puede desenchufar la línea a continuación para ver * todos * eventos durante la ejecución
# # print (f "[evento] autor: {event.author}, type: {type (evento) .__ name__}, final: {event.is_final_esponse ()}, content: {event.content}")

# # Concepto de clave: IS_Final_Response () marca el mensaje final para el giro.
      if event.is_final_response():
          if event.content and event.content.parts:
# # Suponiendo la respuesta de texto en la primera parte
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
# # Agregue más cheques aquí si es necesario (por ejemplo, códigos de error específicos)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

---

** 5 \.Ejecutar la conversación **

Finalmente, probemos nuestra configuración enviando algunas consultas al agente.Envolvemos nuestras llamadas `Async` en una función principal` Async` y la ejecutamos usando `esperanza '.

Mira la salida:

* Ver las consultas de usuario.
* Observe la herramienta `---: get_weather llamada ... ---` registra cuando el agente usa la herramienta.
* Observe las respuestas finales del agente, incluida la forma en que maneja el caso donde los datos meteorológicos no están disponibles (para París).

In [22]:
# Title Ejecute la conversación inicial

# Necesitamos una función asíncrata para esperar a nuestro ayudante de interacción
async def run_conversation():
    await call_agent_async("What is the weather like in London?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("How about Paris?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # Expecting the tool's error message

    await call_agent_async("Tell me the weather in New York",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Ejecute la conversación usando espera en un contexto de asíncrono (como Colab/Jupyter)
await run_conversation()


>>> User Query: What is the weather like in London?


--- Tool: get_weather called for city: London ---
<<< Agent Response: The weather in London is cloudy with a temperature of 15°C.


>>> User Query: How about Paris?


--- Tool: get_weather called for city: Paris ---
<<< Agent Response: I am sorry, I don't have weather information for Paris.


>>> User Query: Tell me the weather in New York


--- Tool: get_weather called for city: New York ---
<<< Agent Response: The weather in New York is sunny with a temperature of 25°C.



---

Felicidades\!Ha construido e interactuado con éxito con su primer agente de ADK.Entiende la solicitud del usuario, utiliza una herramienta para encontrar información y responde adecuadamente en función del resultado de la herramienta.

En el siguiente paso, exploraremos cómo cambiar fácilmente el modelo de lenguaje subyacente que alimenta a este agente.

## Paso 2: Ir múltiples modelos con litellm

En el paso 1, construimos un agente meteorológico funcional impulsado por un modelo de Géminis específico.Si bien son efectivas, las aplicaciones del mundo real a menudo se benefician de la flexibilidad para usar * diferentes * modelos de lenguaje grande (LLM).¿Por qué?

*** Rendimiento: ** Algunos modelos se destacan en tareas específicas (por ejemplo, codificación, razonamiento, escritura creativa).
*** Costo: ** Los diferentes modelos tienen puntos de precio variables.
*** Capacidades: ** Los modelos ofrecen características diversas, tamaños de ventana de contexto y opciones de ajuste fino.
*** Disponibilidad/redundancia: ** Tener alternativas asegura que su aplicación permanezca funcional incluso si un proveedor experimenta problemas.

ADK hace que el cambio entre modelos sea sin costuras a través de su integración con la biblioteca [** litellm **] (https://github.com/berriai/litellm).Litellm actúa como una interfaz consistente para más de 100 LLM diferentes.

** En este paso, lo haremos: **

1. Aprenda a configurar un ADK `agente` para usar modelos de proveedores como OpenAI (GPT) y Anthrope (Claude) utilizando el envoltorio 'litellm`.
2. Definir, configurar (con sus propias sesiones y corredores), e inmediatamente pruebe las instancias de nuestro agente meteorológico, cada uno respaldado por un LLM diferente.
3. Interactuar con estos diferentes agentes para observar variaciones potenciales en sus respuestas, incluso cuando se usa la misma herramienta subyacente.

---

** 1 \.Importar `litellm` **

Importamos esto durante la configuración inicial (Paso 0), pero es el componente clave para el soporte de múltiples modelos:

In [23]:
# Title 1. Importar litellm
from google.adk.models.lite_llm import LiteLlm

** 2 \.Definir y probar agentes multimodelo **

En lugar de pasar solo una cadena de nombre del modelo (que es predeterminada a los modelos Gemini de Google), envolvemos la cadena de identificador del modelo deseada dentro de la clase 'litellm`.

*** Concepto clave: `litellm` Wrapper: ** El` litellm (model = "proveedor/model_name") `La sintaxis le dice a ADK que enrute las solicitudes de este agente a través de la biblioteca litellm al proveedor de modelos especificado.

Asegúrese de haber configurado las claves API necesarias para OpenAI y Anthrope en el paso 0. Usaremos la función `Call_agent_async` (definida anteriormente, que ahora acepta` Runner`, `User_id` y` session_id`) para interactuar con cada agente inmediatamente después de su configuración.

Cada bloque a continuación será:
* Defina el agente utilizando un modelo litellm específico (`model_gpt_4o` o` model_claude_sonnet`).
* Cree un * nuevo, separado * `InMemorySessionService` y Session específicamente para la ejecución de la prueba de ese agente.Esto mantiene las historias de conversación aisladas para esta demostración.
* Cree un `Runner` configurado para el agente específico y su servicio de sesión.
* Llame inmediatamente a `call_agent_async` para enviar una consulta y probar el agente.

** Las mejores prácticas: ** Use constantes para nombres de modelos (como `model_gpt_4o`,` model_claude_sonnet` definido en el paso 0) para evitar errores tipográficos y facilitar el código de administrar.

** Manejo de errores: ** Envolvemos las definiciones del agente en `intento ... excepto 'bloques.Esto evita que la celda de código completa falle si falta una clave API para un proveedor específico o no es válido, lo que permite que el tutorial continúe con los modelos que * están * configurados.

Primero, creemos y probemos el agente usando el GPT-4O de OpenAI.

In [24]:
# title Define and Test GPT Agent

# Asegúrese de que la función 'get_weather' del paso 1 se define en su entorno.
# Asegúrese de que 'call_agent_async' se define desde anteriormente.

# --- Agente usando GPT-4O ---
weather_agent_gpt = None # Initialize to None
runner_gpt = None      # Initialize runner to None

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
# # Cambio de teclas: envuelva el identificador del modelo litellm
        model=LiteLlm(model=MODEL_GPT_4O),
        description="Provides weather information (using GPT-4o).",
        instruction="You are a helpful weather assistant powered by GPT-4o. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Clearly present successful reports or polite error messages based on the tool's output status.",
        tools=[get_weather], # Re-use the same tool
    )
    print(f"Agent '{weather_agent_gpt.name}' created using model '{MODEL_GPT_4O}'.")

# # InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
    session_service_gpt = InMemorySessionService() # Create a dedicated service

# # Definir constantes para identificar el contexto de interacción
    APP_NAME_GPT = "weather_tutorial_app_gpt" # Unique app name for this test
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt" # Using a fixed ID for simplicity

# # Crea la sesión específica donde sucederá la conversación
    session_gpt = session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"Session created: App='{APP_NAME_GPT}', User='{USER_ID_GPT}', Session='{SESSION_ID_GPT}'")

# # Crear un corredor específico para este agente y su servicio de sesión
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,       # Use the specific app name
        session_service=session_service_gpt # Use the specific session service
        )
    print(f"Runner created for agent '{runner_gpt.agent.name}'.")

    # --- Test the GPT Agent ---
    print("\n--- Testing GPT Agent ---")
# # Asegúrese de que Call_agent_async use el corredor correcto, user_id, session_id
    await call_agent_async(query = "What's the weather in Tokyo?",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)

except Exception as e:
    print(f"❌ Could not create or run GPT agent '{MODEL_GPT_4O}'. Check API Key and model name. Error: {e}")


Agent 'weather_agent_gpt' created using model 'openai/gpt-4o-mini'.
Session created: App='weather_tutorial_app_gpt', User='user_1_gpt', Session='session_001_gpt'
Runner created for agent 'weather_agent_gpt'.

--- Testing GPT Agent ---

>>> User Query: What's the weather in Tokyo?
--- Tool: get_weather called for city: Tokyo ---
<<< Agent Response: The weather in Tokyo is currently light rain with a temperature of 18°C.


A continuación, haremos lo mismo por el soneto Claude de Anthrope.

In [25]:
# Title Definir y probar el agente de Claude

# Asegúrese de que la función 'get_weather' del paso 1 se define en su entorno.
# Asegúrese de que 'call_agent_async' se define desde anteriormente.

# --- Agente usando el soneto de Claude ---
weather_agent_claude = None # Initialize to None
runner_claude = None      # Initialize runner to None

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
# # Cambio de teclas: envuelva el identificador del modelo litellm
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="Provides weather information (using Claude Sonnet).",
        instruction="You are a helpful weather assistant powered by Claude Sonnet. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Analyze the tool's dictionary output ('status', 'report'/'error_message'). "
                    "Clearly present successful reports or polite error messages.",
        tools=[get_weather], # Re-use the same tool
    )
    print(f"Agent '{weather_agent_claude.name}' created using model '{MODEL_CLAUDE_SONNET}'.")

# # InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
    session_service_claude = InMemorySessionService() # Create a dedicated service

# # Definir constantes para identificar el contexto de interacción
    APP_NAME_CLAUDE = "weather_tutorial_app_claude" # Unique app name
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude" # Using a fixed ID for simplicity

# # Crea la sesión específica donde sucederá la conversación
    session_claude = session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"Session created: App='{APP_NAME_CLAUDE}', User='{USER_ID_CLAUDE}', Session='{SESSION_ID_CLAUDE}'")

# # Crear un corredor específico para este agente y su servicio de sesión
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,       # Use the specific app name
        session_service=session_service_claude # Use the specific session service
        )
    print(f"Runner created for agent '{runner_claude.agent.name}'.")

# # --- Prueba el agente de Claude ---
    print("\n--- Testing Claude Agent ---")
# # Asegúrese de que Call_agent_async use el corredor correcto, user_id, session_id
    await call_agent_async(query = "Weather in London please.",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

except Exception as e:
    print(f"❌ Could not create or run Claude agent '{MODEL_CLAUDE_SONNET}'. Check API Key and model name. Error: {e}")

Agent 'weather_agent_claude' created using model 'anthropic/claude-3-sonnet-20240229'.
Session created: App='weather_tutorial_app_claude', User='user_1_claude', Session='session_001_claude'
Runner created for agent 'weather_agent_claude'.

--- Testing Claude Agent ---

>>> User Query: Weather in London please.
--- Tool: get_weather called for city: London ---
<<< Agent Response: The weather report for London is:

It's cloudy in London with a temperature of 15°C.


Observe la salida cuidadosamente desde ambos bloques de código.Deberías ver:

1. Cada agente (`Weather_agent_gpt`,` Weather_agent_claude`) se crea con éxito (si las teclas API son válidas).
2. Una sesión dedicada y un corredor están configurados para cada uno.
3. Cada agente identifica correctamente la necesidad de usar la herramienta `get_weather` al procesar la consulta (verá la herramienta` ---: get_weather llamada ... --- `log).
4. La lógica de la herramienta * subyacente * sigue siendo idéntica, siempre devuelve nuestros datos simulados.
5. Sin embargo, la respuesta textual final ** generada por cada agente podría diferir ligeramente en el fraseo, tono o formato.Esto se debe a que el mensaje de instrucción es interpretado y ejecutado por diferentes LLM (GPT-4O vs. Claude sonnet).

Este paso demuestra la potencia y la flexibilidad de ADK + litellm.Puede experimentar e implementar fácilmente agentes utilizando varios LLM mientras mantiene consistente su lógica de aplicación central (herramientas, estructura de agentes fundamentales).

¡En el siguiente paso, iremos más allá de un solo agente y construiremos un equipo pequeño donde los agentes puedan delegar tareas entre sí!

---

## Paso 3: Construyendo un equipo de agente \- Delegación para saludos y despedidas

En los pasos 1 y 2, construimos y experimentamos con un solo agente enfocado únicamente en las búsquedas meteorológicas.Si bien es efectivo para su tarea específica, las aplicaciones del mundo real a menudo implican manejar una variedad más amplia de interacciones de usuario.* Podríamos * seguir agregando más herramientas e instrucciones complejas a nuestro agente meteorológico único, pero esto puede volverse inmanejable y menos eficiente.

Un enfoque más robusto es construir un ** equipo de agente **.Esto implica:

1. Creación de múltiples agentes especializados **, cada uno diseñado para una capacidad específica (por ejemplo, uno para clima, uno para saludos, uno para cálculos).
2. Designando un agente raíz ** ** (o orquestador) que recibe la solicitud inicial del usuario.
3. Habilitar al agente raíz para ** Delegar ** La solicitud al sub-agente especializado más apropiado en función de la intención del usuario.

** ¿Por qué construir un equipo de agente? **

*** Modularidad: ** Más fácil de desarrollar, probar y mantener agentes individuales.
*** Especialización: ** Cada agente se puede ajustar (instrucciones, elección del modelo) para su tarea específica.
*** Escalabilidad: ** Más simple agregar nuevas capacidades agregando nuevos agentes.
*** Eficiencia: ** Permite usar modelos potencialmente más simples/más baratos para tareas más simples (como saludos).

** En este paso, lo haremos: **

1. Defina herramientas simples para manejar saludos (`say_hello`) y despedidas (` say_goodbye`).
2. Cree dos nuevos sub-agentes especializados: `Saluding_agent` y` Farewell_agent`.
3. Actualice nuestro agente meteorológico principal (`Weather_Agent_V2`) para actuar como el ** Root Agent **.
4. Configure el agente raíz con sus subcásgenes, habilitando ** la delegación automática **.
5. Pruebe el flujo de delegación enviando diferentes tipos de solicitudes al agente raíz.

---

** 1 \.Definir herramientas para sub-agentes **

Primero, creemos las funciones simples de Python que servirán como herramientas para nuestros nuevos agentes especializados.Recuerde, las documentos claros son vitales para los agentes que los usarán.

In [26]:
# Title Definir herramientas para agentes de saludo y despedida

# Asegúrese de que 'get_weather' del paso 1 esté disponible si ejecuta este paso de forma independiente.
# Def get_weather (ciudad: str) -> dict: ... (del paso 1)

def say_hello(name: str = "there") -> str:
    """Provides a simple greeting, optionally addressing the user by name.

    Args:
        name (str, optional): The name of the person to greet. Defaults to "there".

    Returns:
        str: A friendly greeting message.
    """
    print(f"--- Tool: say_hello called with name: {name} ---")
    return f"Hello, {name}!"

def say_goodbye() -> str:
    """Provides a simple farewell message to conclude the conversation."""
    print(f"--- Tool: say_goodbye called ---")
    return "Goodbye! Have a great day."

print("Greeting and Farewell tools defined.")

# Testimas de autocomprobación opcional
print(say_hello("Alice"))
print(say_goodbye())

Greeting and Farewell tools defined.
--- Tool: say_hello called with name: Alice ---
Hello, Alice!
--- Tool: say_goodbye called ---
Goodbye! Have a great day.


---

** 2 \.Definir los sub-agentes (saludo y despedida) **

Ahora, cree las instancias de `agente` para nuestros especialistas.Observe su 'instrucción' altamente enfocada y, críticamente, su clara 'descripción'.La `descripción` es la información principal que * Root Agent * usa para decidir * cuando * delegar a estos subcacios.

¡Incluso podemos usar diferentes LLM para estos subgestados \!Asignemos GPT-4O al agente de saludo y mantengamos al agente de despedida usando GPT-4O también (podría cambiar fácilmente uno a Claude o Gemini si lo desea y las teclas API están establecidas).

** Las mejores prácticas: ** Los campos de `descripción` sub-agentes deben resumir de manera precisa y concisa su capacidad específica.Esto es crucial para una delegación automática efectiva.

** Las mejores prácticas: ** Los campos de `instrucciones` sub-agentes deben adaptarse a su alcance limitado, diciéndoles exactamente qué hacer y*qué no*hacer (por ejemplo," Su*solo*tarea es ... ").

In [27]:
# Title Defina sub-agentes de saludo y despedida

# Asegúrese de que se importe litellm y las teclas API se establezcan (desde el paso 0/2)
# de google.adk.models.lite_llm import litellm
# Model_gpt_4o, model_claude_sonnet, etc. debe definirse

# --- Agente de saludo ---
greeting_agent = None
try:
    greeting_agent = Agent(
# Uso de un modelo potencialmente diferente/más barato para una tarea simple
        model=LiteLlm(model=MODEL_GPT_4O),
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting to the user. "
                    "Use the 'say_hello' tool to generate the greeting. "
                    "If the user provides their name, make sure to pass it to the tool. "
                    "Do not engage in any other conversation or tasks.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.", # Crucial for delegation
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' created using model '{MODEL_GPT_4O}'.")
except Exception as e:
    print(f"❌ Could not create Greeting agent. Check API Key ({MODEL_GPT_4O}). Error: {e}")

# --- Agente de despedida ---
farewell_agent = None
try:
    farewell_agent = Agent(
# Puede usar el mismo modelo o un modelo diferente
        model=LiteLlm(model=MODEL_GPT_4O), # Sticking with GPT for this example
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message. "
                    "Use the 'say_goodbye' tool when the user indicates they are leaving or ending the conversation "
                    "(e.g., using words like 'bye', 'goodbye', 'thanks bye', 'see you'). "
                    "Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.", # Crucial for delegation
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' created using model '{MODEL_GPT_4O}'.")
except Exception as e:
    print(f"❌ Could not create Farewell agent. Check API Key ({MODEL_GPT_4O}). Error: {e}")

✅ Agent 'greeting_agent' created using model 'openai/gpt-4o-mini'.
✅ Agent 'farewell_agent' created using model 'openai/gpt-4o-mini'.


---

** 3 \.Defina el agente raíz (agente meteorológico v2) con sub-agentes **

Ahora, actualizamos nuestro `Weather_agent`.Los cambios clave son:

* Agregar el parámetro `sub_agents`: pasamos una lista que contiene las instancias` Saluding_Agent` y `Farewell_agent` que acabamos de crear.
* Actualización de la `instrucción`: le decimos explícitamente al agente raíz * sobre * sus sub-agentes y * cuando * debe delegarles tareas.

** Concepto clave: Delegación automática (flujo automático) ** Al proporcionar la lista de `sub_agents ', ADK habilita la delegación automática.Cuando el agente raíz recibe una consulta de usuario, su LLM considera no solo sus propias instrucciones y herramientas sino también la 'descripción' de cada subgent.Si el LLM determina que una consulta se alinea mejor con la capacidad descrita de un subgente (por ejemplo, "maneja saludos simples"), generará automáticamente una acción interna especial para * transferir control * a ese subgent para ese turno.El subagente luego procesa la consulta utilizando su propio modelo, instrucciones y herramientas.

** Las mejores prácticas: ** Asegúrese de que las instrucciones del agente raíz guíen claramente sus decisiones de delegación.Mencione los subgestados por nombre y describa las condiciones bajo las cuales debe ocurrir la delegación.

In [28]:
# Title Defina el agente raíz con los subcásgenes

# Asegúrese de que los subgestados se creen con éxito antes de definir el agente raíz.
# También asegúrese de definir la herramienta original 'get_weather'.
root_agent = None
runner_root = None # Initialize runner

if greeting_agent and farewell_agent and 'get_weather' in globals():
# # Usemos un modelo Géminis capaz para que el agente de la raíz maneje la orquestación
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2", # Give it a new version name
        model=root_agent_model,
        description="The main coordinator agent. Handles weather requests and delegates greetings/farewells to specialists.",
        instruction="You are the main Weather Agent coordinating a team. Your primary responsibility is to provide weather information. "
                    "Use the 'get_weather' tool ONLY for specific weather requests (e.g., 'weather in London'). "
                    "You have specialized sub-agents: "
                    "1. 'greeting_agent': Handles simple greetings like 'Hi', 'Hello'. Delegate to it for these. "
                    "2. 'farewell_agent': Handles simple farewells like 'Bye', 'See you'. Delegate to it for these. "
                    "Analyze the user's query. If it's a greeting, delegate to 'greeting_agent'. If it's a farewell, delegate to 'farewell_agent'. "
                    "If it's a weather request, handle it yourself using 'get_weather'. "
                    "For anything else, respond appropriately or state you cannot handle it.",
        tools=[get_weather], # Root agent still needs the weather tool for its core task
# # Cambio de teclas: ¡Enlace los sub-agentes aquí!
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ Root Agent '{weather_agent_team.name}' created using model '{root_agent_model}' with sub-agents: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ Cannot create root agent because one or more sub-agents failed to initialize or 'get_weather' tool is missing.")
    if not greeting_agent: print(" - Greeting Agent is missing.")
    if not farewell_agent: print(" - Farewell Agent is missing.")
    if 'get_weather' not in globals(): print(" - get_weather function is missing.")



✅ Root Agent 'weather_agent_v2' created using model 'gemini-2.0-flash-exp' with sub-agents: ['greeting_agent', 'farewell_agent']


---

** 4 \.Interactuar con el equipo del agente **

Ahora que hemos definido nuestro agente raíz (`weather_agent_team`- *Nota: Asegúrese de que este nombre de variable coincida con el definido en el bloque de código anterior, probablemente`# @title defina el agente raíz con los subgentadores ', que podría haberlo nombrado `root_agent` *) con sus sub-agentes especializados, prueba el mecanismo de delegación.

El siguiente bloque de código:

1. Defina una función `async`` run_team_conversation`.
2. Dentro de esta función, cree un * nuevo y dedicado * `InMemorySessionService` y una sesión específica (` session_001_agent_team`) solo para esta ejecución de prueba.Esto aísla el historial de conversación para probar la dinámica del equipo.
3. Cree un `Runner` (` Runner_agent_Team`) configurado para usar nuestro 'Weather_agent_Team` (el agente raíz) y el servicio de sesión dedicado.
4. Use nuestra función actualizada de `call_agent_async` para enviar diferentes tipos de consultas (saludo, solicitud de clima, despedida) al 'runner_agent_team`.Pasamos explícitamente el corredor, ID de usuario y ID de sesión para esta prueba específica.
5. Ejecute inmediatamente la función `run_team_conversation`.

Esperamos el siguiente flujo:

1. El "¡Hola!"La consulta va a `RUNNER_AGENT_TEAM`.
2. El agente raíz (`weather_agent_team`) lo recibe y, según sus instrucciones y la descripción de` saludo_agent`, delega la tarea.
3. `Saluding_Agent` maneja la consulta, llama a su herramienta` say_hello` y genera la respuesta.
4. El "¿Cuál es el clima en Nueva York?"La consulta es * no * delegada y es manejada directamente por el agente raíz utilizando su herramienta 'get_weather`.
5. El "¡Gracias, adiós!"La consulta se delega a la `Farwell_agent`, que utiliza su herramienta 'say_goodbye`.

In [29]:
# Title interactúa con el equipo de agentes

# Asegúrese de que se define el agente raíz (por ejemplo, 'weather_agent_team' o 'root_agent' de la celda anterior).
# Asegúrese de definir la función Call_agent_async.

# Compruebe si la variable de agente raíz existe antes de definir la función de conversación
root_agent_var_name = 'root_agent' # Default name from Step 3 guide
if 'weather_agent_team' in globals(): # Check if user used this name instead
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ Root agent ('root_agent' or 'weather_agent_team') not found. Cannot define run_team_conversation.")
# # Asigne un valor ficticio para evitar NameError más tarde si el bloque de código se ejecuta de todos modos
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- Testing Agent Team Delegation ---")
# # InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
        session_service = InMemorySessionService()

# # Definir constantes para identificar el contexto de interacción
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team" # Using a fixed ID for simplicity

# # Crea la sesión específica donde sucederá la conversación
        session = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# # --- Obtenga el objeto de agente raíz real ---
# # Use el nombre de la variable determinado
        actual_root_agent = globals()[root_agent_var_name]

# # Crear un corredor específico para esta prueba de equipo de agente
        runner_agent_team = Runner(
            agent=actual_root_agent, # Use the root agent object
            app_name=APP_NAME,       # Use the specific app name
            session_service=session_service # Use the specific session service
            )
# # Declaración de impresión corregida para mostrar el nombre del agente raíz real
        print(f"Runner created for agent '{actual_root_agent.name}'.")

# # Siempre interactúa a través del corredor del agente raíz, pasando las identificaciones correctas
        await call_agent_async(query = "Hello there!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "What is the weather in New York?",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "Thanks, bye!",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

# # Ejecutar la conversación
# # NOTA: ¡Esto puede requerir claves API para los modelos utilizados por la raíz y los subcibentes!
    await run_team_conversation()
else:
    print("\n⚠️ Skipping agent team conversation as the root agent was not successfully defined in the previous step.")



--- Testing Agent Team Delegation ---
Session created: App='weather_tutorial_agent_team', User='user_1_agent_team', Session='session_001_agent_team'
Runner created for agent 'weather_agent_v2'.

>>> User Query: Hello there!


--- Tool: say_hello called with name: there ---


<<< Agent Response: Hello, there!

>>> User Query: What is the weather in New York?


--- Tool: get_weather called for city: New York ---
<<< Agent Response: The weather in New York is sunny with a temperature of 25°C.


>>> User Query: Thanks, bye!


--- Tool: say_goodbye called ---
<<< Agent Response: Goodbye! Have a great day.


---

Mire de cerca los registros de salida, especialmente la herramienta `---: ... llamada ---` Mensajes.Deberías observar:

* Para "¡Hola!", Se llamó a la herramienta `say_hello` (que indica` saludo_agent` lo manejó).
* Para "¿Cuál es el clima en Nueva York?", Se llamó la herramienta `get_weather` (indicando que el agente raíz lo manejó).
* Por "¡Gracias, adiós!", Se llamó la herramienta `say_goodbye` (que indica` Farewell_agent` lo manejó).

¡Esto confirma la delegación automática exitosa **!El agente raíz, guiado por sus instrucciones y la 'descripción' de su `sub_agents`, enrutó correctamente las solicitudes de usuario al agente especializado apropiado dentro del equipo.

Ahora ha estructurado su aplicación con múltiples agentes de colaboración.Este diseño modular es fundamental para construir sistemas de agentes más complejos y capaces.En el siguiente paso, daremos a nuestros agentes la capacidad de recordar información en los giros utilizando el estado de la sesión.

## Paso 4: Agregar memoria y personalización con el estado de sesión

Hasta ahora, nuestro equipo de agentes puede manejar diferentes tareas a través de la delegación, pero cada interacción comienza fresca: los agentes no tienen memoria de conversaciones pasadas o preferencias de usuario dentro de una sesión.Para crear experiencias más sofisticadas y conscientes del contexto, los agentes necesitan ** memoria **.ADK proporciona esto a través del estado de la sesión ** **.

** ¿Qué es el estado de la sesión? **

* Es un Diccionario de Python (`session.state`) vinculado a una sesión de usuario específica (identificada por` app_name`, `user_id`,` session_id`).
* Persiste información * en múltiples giros conversacionales * dentro de esa sesión.
* Los agentes y herramientas pueden leer y escribir en este estado, lo que les permite recordar detalles, adaptar el comportamiento y personalizar las respuestas.

** Cómo los agentes interactúan con el estado: **

1. ** `ToolContext` (método primario): ** Las herramientas pueden aceptar un objeto` ToolContext` (proporcionado automáticamente por ADK si se declara como el último argumento).Este objeto brinda acceso directo al estado de sesión a través de `tool_context.state`, permitiendo herramientas para leer las preferencias o guardar resultados * durante * ejecución.
2. ** `Output_Key` (respuesta de agente automático): ** Un` agente` se puede configurar con un `output_key =" your_key "`.ADK guardará automáticamente la respuesta textual final del agente para convertir un convertido en `session.state [" your_key "]`.

** En este paso, mejoraremos nuestro equipo de botes meteorológicos por: **

1. Uso de un ** nuevo ** `InMemorySessionService` para demostrar el estado de forma aislada.
2. Inicialización del estado de la sesión con una preferencia del usuario por `temperatura_unit`.
3. Creación de una versión de estado de estado de la herramienta meteorológica (`get_weather_stateful`) que lee esta preferencia a través de` ToolContext` y ajusta su formato de salida (Celsius/Fahrenheit).
4. Actualización del agente raíz para usar esta herramienta con estado y configurarla con un `output_key` para guardar automáticamente su informe meteorológico final en el estado de la sesión.
5. Ejecutar una conversación para observar cómo el estado inicial afecta la herramienta, cómo los cambios en el estado manual alteran el comportamiento posterior y cómo `` `` `ones persiste '' persiste la respuesta del agente.

---

** 1 \.Inicializar el nuevo servicio de sesión y el estado **

Para demostrar claramente la gestión del estado sin interferencia de pasos anteriores, instanciaremos un nuevo 'InMemorySessionService'.También crearemos una sesión con un estado inicial que define la unidad de temperatura preferida del usuario.

In [30]:
# Title 1. Inicializar un nuevo servicio de sesión y estado

# Importar componentes de sesión necesarios
from google.adk.sessions import InMemorySessionService

# Cree una nueva instancia de servicio de sesión para esta demostración estatal
session_service_stateful = InMemorySessionService()
print("✅ New InMemorySessionService created for state demonstration.")

# Defina una nueva identificación de sesión para esta parte del tutorial
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# Definir datos de estado iniciales: el usuario prefiere Celsius inicialmente
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# Crear la sesión, proporcionando el estado inicial
session_stateful = session_service_stateful.create_session(
    app_name=APP_NAME, # Use the consistent app name
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< Initialize state during creation
)
print(f"✅ Session '{SESSION_ID_STATEFUL}' created for user '{USER_ID_STATEFUL}'.")

# Verificar que el estado inicial se estableció correctamente
retrieved_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- Initial Session State ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("Error: Could not retrieve session.")

✅ New InMemorySessionService created for state demonstration.
✅ Session 'session_state_demo_001' created for user 'user_state_demo'.

--- Initial Session State ---
{'user_preference_temperature_unit': 'Celsius'}


---

** 2 \.Crear herramienta meteorológica consciente del estado (`get_weather_stateful`) **

Ahora, creamos una nueva versión de la herramienta meteorológica.Su característica clave es aceptar `Tool_Context: ToolContext` que le permite acceder a` tool_context.state`.Leerá el `user_preference_temperature_unit` y formateará la temperatura en consecuencia.


*** Concepto clave: `ToolContext` ** Este objeto es el puente que permite que la lógica de su herramienta interactúe con el contexto de la sesión, incluida la lectura y la escritura de las variables de estado.ADK lo inyecta automáticamente si se define como el último parámetro de la función de su herramienta.


*** Mejor práctica: ** Al leer desde el estado, use `diccionario.get ('key', default_value)` para manejar casos donde la clave aún no existirá, asegurando que su herramienta no se bloquee.

In [31]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """Retrieves weather, converts temp unit based on session state."""
    print(f"--- Tool: get_weather_stateful called for {city} ---")

# # --- Leer preferencia del estado ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius") # Default to Celsius
    print(f"--- Tool: Reading state 'user_preference_temperature_unit': {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")

# # Datos meteorológicos simulados (siempre almacenados en Celsius internamente)
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "sunny"},
        "london": {"temp_c": 15, "condition": "cloudy"},
        "tokyo": {"temp_c": 18, "condition": "light rain"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

# # Temperatura de formato basada en la preferencia de estado
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32 # Calculate Fahrenheit
            temp_unit = "°F"
        else: # Default to Celsius
            temp_value = temp_c
            temp_unit = "°C"

        report = f"The weather in {city.capitalize()} is {condition} with a temperature of {temp_value:.0f}{temp_unit}."
        result = {"status": "success", "report": report}
        print(f"--- Tool: Generated report in {preferred_unit}. Result: {result} ---")

# # Ejemplo de redacción de regreso al estado (opcional para esta herramienta)
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- Tool: Updated state 'last_city_checked_stateful': {city} ---")

        return result
    else:
# # Manejar la ciudad no encontrada
        error_msg = f"Sorry, I don't have weather information for '{city}'."
        print(f"--- Tool: City '{city}' not found. ---")
        return {"status": "error", "error_message": error_msg}

print("✅ State-aware 'get_weather_stateful' tool defined.")


✅ State-aware 'get_weather_stateful' tool defined.


---

** 3 \.Redefinir los subgestados y actualizar el agente raíz **

Para garantizar que este paso sea autónomo y se construye correctamente, primero redefinimos el `Saluding_Agent` y` Farewell_agent` exactamente como estaban en el paso 3 \.Luego, definimos nuestro nuevo agente raíz (`weather_agent_v4_stateful`):

* Utiliza la nueva herramienta `get_weather_stateful`.
* Incluye los sub-agentes de saludo y despedida para la delegación.
*** Crucialmente **, establece `output_key =" last_weather_report "` que guarda automáticamente su respuesta climática final al estado de sesión.

In [32]:
# Title 3. Redefine sub-agentes y actualice Root Agent con Output_Key

# Asegurar las importaciones necesarias: agente, litellm, corredor
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
# Asegúrese de que las herramientas 'say_hello', 'say_goodbye' se definan (del paso 3)
# Asegurar constantes de modelo Model_gpt_4o, model_gemini_2_5_pro etc.

# --- Redefine Greeting Agent (from Step 3) ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Error: {e}")

# --- Redefine el agente de despedida (del paso 3) ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Error: {e}")

# --- Define the Updated Root Agent ---
root_agent_stateful = None
runner_root_stateful = None # Initialize runner

# Verifique los requisitos previos antes de crear el agente raíz
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH # Choose orchestration model

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful", # New version name
        model=root_agent_model,
        description="Main agent: Provides weather (state-aware unit), delegates greetings/farewells, saves report to state.",
        instruction="You are the main Weather Agent. Your job is to provide weather using 'get_weather_stateful'. "
                    "The tool will format the temperature based on user preference stored in state. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather_stateful], # Use the state-aware tool
        sub_agents=[greeting_agent, farewell_agent], # Include sub-agents
        output_key="last_weather_report" # <<< Auto-save agent's final weather response
    )
    print(f"✅ Root Agent '{root_agent_stateful.name}' created using stateful tool and output_key.")

# # --- Crear corredor para este agente raíz y nuevo servicio de sesión ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # Use the NEW stateful session service
    )
    print(f"✅ Runner created for stateful root agent '{runner_root_stateful.agent.name}' using stateful session service.")

else:
    print("❌ Cannot create stateful root agent. Prerequisites missing.")
    if not greeting_agent: print(" - greeting_agent definition missing.")
    if not farewell_agent: print(" - farewell_agent definition missing.")
    if 'get_weather_stateful' not in globals(): print(" - get_weather_stateful tool missing.")


✅ Agent 'greeting_agent' redefined.
✅ Agent 'farewell_agent' redefined.
✅ Root Agent 'weather_agent_v4_stateful' created using stateful tool and output_key.
✅ Runner created for stateful root agent 'weather_agent_v4_stateful' using stateful session service.


---

** 4 \.Interactuar y probar el flujo de estado **

Ahora, ejecutemos una conversación diseñada para probar las interacciones de estado utilizando el 'Runner_Root_Stateful` (asociado con nuestro agente con estado y el `session_service_stateful`).Usaremos la función `call_agent_async` definida anteriormente, asegurando que pasemos el corredor correcto, la identificación de usuario (` user_id_stateful`) e ID de sesión (`session_id_stateful`).

El flujo de conversación será:

1.  **Check weather (London):** The `get_weather_stateful` tool should read the initial "Celsius" preference from the session state initialized in Section 1. The root agent's final response (the weather report in Celsius) should get saved to `state['last_weather_report']` via the `output_key` configuration.
2. ** Estado de actualización manual: ** modificaremos directamente*el estado almacenado dentro de la instancia `inMemorySessionService` (` session_service_stateful`).
*** ¿Por qué la modificación directa? ** El `session_service.get_session ()` El método devuelve un*copia*de la sesión.Modificar esa copia no afectaría el estado utilizado en las ejecuciones de agentes posteriores.Para este escenario de prueba con `InMemorySessionService`, accedemos al diccionario interno` sessions` para cambiar el valor de estado * real * almacenado * para `user_preference_temperature_unit` a" Fahrenheit ".*Nota: En aplicaciones reales, los cambios de estado generalmente se activan por herramientas o lógica de agente que devuelve `eventactions (state_delta = ...)`, no actualizaciones manuales directas.*
3. ** Compruebe el clima nuevamente (Nueva York): ** La herramienta `get_weather_stateful` ahora debería leer la preferencia actualizada de" fahrenheit "del estado y convertir la temperatura en consecuencia.La respuesta * nueva * del agente raíz (clima en fahrenheit) sobrescribirá el valor anterior en `state ['last_weather_report']` debido a la 'salida_key`.
4. ** Salude al agente: ** Verifique que la delegación al `Saluding_Agent` todavía funciona correctamente junto con las operaciones con estado.Esta interacción se convertirá en la * Última * respuesta guardada por `output_key` en esta secuencia específica.
5. ** Inspeccione el estado final: ** Después de la conversación, recuperamos la sesión por última vez (obtener una copia) e imprimimos su estado para confirmar el `user_preference_temperature_unit` es de hecho" fahrenheit ", observe el valor final guardado por` £key` (que será el saludo en esta ejecución) y ver el `` `Last_City_Checked Fulio` `GOOUT_KEY` (que será el saludo en esta ejecución), y ver el` `Last_City_Checked Fule` `GOOUT_KEY` (que será el saludo en esta ejecución), y ver el` `Last_City_Checked Fule '` `GOOUT_KEY` (que será el saludo en esta ejecución) y ver el` `` `` `` `` `` `` `` `Last_City_Checked Fulio' '.

In [33]:
# Title 4. Interactuar para probar el flujo de estado y la salida_key

# Asegúrese de que el corredor estatal (runner_root_stateful) esté disponible en la celda anterior
# Asegúrese de que Call_agent_async, user_id_stateful, session_id_stateful, app_name se definan

if 'runner_root_stateful' in globals() and runner_root_stateful:
  async def run_stateful_conversation():
      print("\n--- Testing State: Temp Unit Conversion & output_key ---")

# # 1. Verifique el clima (usa el estado inicial: Celsius)
      print("--- Turn 1: Requesting weather in London (expect Celsius) ---")
      await call_agent_async(query= "What's the weather in London?",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

# # 2. Actualice manualmente la preferencia de estado a Fahrenheit: modifique directamente el almacenamiento
      print("\n--- Manually Updating State: Setting unit to Fahrenheit ---")
      try:
# # Acceda al almacenamiento interno directamente: esto es específico de InMemorySessionService para pruebas
          stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
          stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
# # Opcional: es posible que desee actualizar la marca de tiempo también si alguna lógica depende de ella
# # Tiempo de importación
# # stored_session.last_update_time = time.time ()
          print(f"--- Stored session state updated. Current 'user_preference_temperature_unit': {stored_session.state['user_preference_temperature_unit']} ---")
      except KeyError:
          print(f"--- Error: Could not retrieve session '{SESSION_ID_STATEFUL}' from internal storage for user '{USER_ID_STATEFUL}' in app '{APP_NAME}' to update state. Check IDs and if session was created. ---")
      except Exception as e:
           print(f"--- Error updating internal session state: {e} ---")

# # 3. Verifique el clima nuevamente (la herramienta ahora debe usar Fahrenheit)
# # Esto también actualizará 'last_weather_report' a través de output_key
      print("\n--- Turn 2: Requesting weather in New York (expect Fahrenheit) ---")
      await call_agent_async(query= "Tell me the weather in New York.",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

# # 4. Pruebe la delegación básica (aún debería funcionar)
# # Esto actualizará 'last_weather_report' nuevamente, sobrescribiendo el informe meteorológico de Nueva York
      print("\n--- Turn 3: Sending a greeting ---")
      await call_agent_async(query= "Hi!",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

# # Ejecutar la conversación
  await run_stateful_conversation()

# # Inspeccione el estado de la sesión final después de la conversación
  print("\n--- Inspecting Final Session State ---")
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id= USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print(f"Final Preference: {final_session.state.get('user_preference_temperature_unit')}")
      print(f"Final Last Weather Report (from output_key): {final_session.state.get('last_weather_report')}")
      print(f"Final Last City Checked (by tool): {final_session.state.get('last_city_checked_stateful')}")
# # Imprimir estado completo para una vista detallada
# # print (f "estado completo: {final_session.state}")
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping state test conversation. Stateful root agent runner ('runner_root_stateful') is not available.")


--- Testing State: Temp Unit Conversion & output_key ---
--- Turn 1: Requesting weather in London (expect Celsius) ---

>>> User Query: What's the weather in London?


--- Tool: get_weather_stateful called for London ---
--- Tool: Reading state 'user_preference_temperature_unit': Celsius ---
--- Tool: Generated report in Celsius. Result: {'status': 'success', 'report': 'The weather in London is cloudy with a temperature of 15°C.'} ---
--- Tool: Updated state 'last_city_checked_stateful': London ---
<<< Agent Response: The weather in London is cloudy with a temperature of 15°C.


--- Manually Updating State: Setting unit to Fahrenheit ---
--- Stored session state updated. Current 'user_preference_temperature_unit': Fahrenheit ---

--- Turn 2: Requesting weather in New York (expect Fahrenheit) ---

>>> User Query: Tell me the weather in New York.


--- Tool: get_weather_stateful called for New York ---
--- Tool: Reading state 'user_preference_temperature_unit': Fahrenheit ---
--- Tool: Generated report in Fahrenheit. Result: {'status': 'success', 'report': 'The weather in New york is sunny with a temperature of 77°F.'} ---
--- Tool: Updated state 'last_city_checked_stateful': New York ---
<<< Agent Response: The weather in New york is sunny with a temperature of 77°F.


--- Turn 3: Sending a greeting ---

>>> User Query: Hi!


--- Tool: say_hello called with name: there ---
<<< Agent Response: Hello, there!


--- Inspecting Final Session State ---
Final Preference: Fahrenheit
Final Last Weather Report (from output_key): Hello, there!

Final Last City Checked (by tool): New York


---

Al revisar el flujo de conversación y la impresión de estado de sesión final, puede confirmar:

*** Estado Leer: ** La herramienta meteorológica (`get_weather_stateful`) lea correctamente` user_preference_temperature_unit` del estado, inicialmente usando "Celsius" para Londres.
*** Actualización de estado: ** La modificación directa cambió con éxito la preferencia almacenada a "Fahrenheit".
*** Estado de lectura (actualizado): ** La herramienta posteriormente leyó "Fahrenheit" cuando se le pidió el clima de Nueva York y realizó la conversión.
*** Escritura de estado de la herramienta: ** La herramienta escribió con éxito el `last_city_checked_stateful` (" Nueva York "después de la segunda verificación del clima) en el estado a través de` tool_context.state`.
*** Delegación: ** La delegación al `Saluding_Agent` para" ¡Hola! "funcionó correctamente incluso después de las modificaciones de estado.
*** `Output_Key`: ** El` output_key = "last_weather_rePort" `guardó con éxito la respuesta*final*final*del agente raíz para*cada turno*donde el agente raíz fue el que finalmente respondió.En esta secuencia, la última respuesta fue el saludo ("¡Hola, allí!"), De modo que sobrescribió el informe meteorológico en la clave del estado.
*** Estado final: ** La verificación final confirma la preferencia persistida como "Fahrenheit".

Ahora ha integrado con éxito el estado de sesión para personalizar el comportamiento de los agentes utilizando el estado `toolcontext`, manipulado manualmente para probar 'inMemorySessionService`, y observó cómo' Output_Key` proporciona un mecanismo simple para guardar la última respuesta del agente al estado.Esta comprensión fundamental de la gestión del estado es clave, ya que procedemos a implementar barandillas de seguridad utilizando devoluciones de llamada en los próximos pasos.

---

## Paso 5: Agregar seguridad \- Entrada GuardRail con `antes_model_callback`

Nuestro equipo de agentes se está volviendo más capaz, recordando las preferencias y el uso de herramientas de manera efectiva.Sin embargo, en los escenarios del mundo real, a menudo necesitamos mecanismos de seguridad para controlar el comportamiento del agente * antes de * las solicitudes potencialmente problemáticas incluso alcanzan el modelo de lenguaje grande (LLM).

ADK proporciona ** devoluciones de llamada **: funciones que le permiten conectarse a puntos específicos en el ciclo de vida de ejecución del agente.El `antes_model_callback` es particularmente útil para la seguridad de la entrada.

** ¿Qué es `antes_model_callback`? **

* Es una función de Python que define que ADK ejecuta * justo antes de * un agente envía su solicitud compilada (incluido el historial de conversación, las instrucciones y el último mensaje de usuario) al LLM subyacente.
*** Propósito: ** Inspeccionar la solicitud, modificarla si es necesario o bloquearla por completo en base a reglas predefinidas.

** Casos de uso comunes: **

*** Validación/filtrado de entrada: ** Compruebe si la entrada del usuario cumple con los criterios o contiene contenido no permitido (como PII o palabras clave).
*** Las barandillas: ** Evite que las solicitudes dañinas, fuera de tema o que violen las políticas sean procesadas por el LLM.
*** Modificación de indicación dinámica: ** Agregue información oportuna (por ejemplo, desde el estado de la sesión) al contexto de solicitud de LLM justo antes de enviar.

** Cómo funciona: **

1. Defina una función que acepte `callback_context: llameBackContext` y` llm_request: llmRequest`.
* `Callback_context`: proporciona acceso a la información del agente, estado de sesión (` callback_context.state`), etc.
* `llm_request`: contiene la carga útil completa destinada al LLM (` contenido`, `config`).
2. Dentro de la función:
*** Inspeccionar: ** Examine `llm_request.contents` (especialmente el último mensaje de usuario).
*** modificar (usar precaución): ** usted*puede*cambiar partes de `llm_request`.
*** Bloque (GuardRail): ** Devuelve un objeto `llmResponse`.ADK devolverá esta respuesta de inmediato, * omitiendo * la llamada LLM para ese turno.
*** Permitir: ** Return `None`.ADK procede a llamar al LLM con la solicitud (potencialmente modificada).

** En este paso, lo haremos: **

1. Defina una función `antes_model_callback` (` block_keyword_guardrail`) que verifica la entrada del usuario para una palabra clave específica ("bloque").
2. Actualice nuestro agente raíz con estado (`Weather_agent_V4_Stateful` del paso 4 \) para usar esta devolución de llamada.
3. Cree un nuevo corredor asociado con este agente actualizado pero utilizando el * mismo servicio de sesión con estado * para mantener la continuidad del estado.
4. Pruebe la barandilla enviando solicitudes normales y que contienen palabras clave.

---

** 1 \.Definir la función de devolución de llamada de la barandilla **

Esta función inspeccionará el último mensaje de usuario dentro del contenido 'LLM_REQUEST`.Si encuentra "bloqueo" (insensible al caso), construye y devuelve un `llmResponse` para bloquear el flujo;De lo contrario, devuelve `Ninguno`.

In [34]:
# Title 1. Defina la barandilla de guía anterior_model_callback

# Asegúrese de que las importaciones necesarias estén disponibles
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # For creating response content
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    Inspects the latest user message for 'BLOCK'. If found, blocks the LLM call
    and returns a predefined LlmResponse. Otherwise, returns None to proceed.
    """
    agent_name = callback_context.agent_name # Get the name of the agent whose model call is being intercepted
    print(f"--- Callback: block_keyword_guardrail running for agent: {agent_name} ---")

# # Extraer el texto del último mensaje de usuario en el historial de solicitudes
    last_user_message_text = ""
    if llm_request.contents:
# # Encuentra el mensaje más reciente con el rol 'Usuario'
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
# # Suponiendo que el texto esté en la primera parte para simplificar
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # Found the last user message text

    print(f"--- Callback: Inspecting last user message: '{last_user_message_text[:100]}...' ---") # Log first 100 chars

# # --- Lógica de barandilla ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # Case-insensitive check
        print(f"--- Callback: Found '{keyword_to_block}'. Blocking LLM call! ---")
# # Opcionalmente, establezca una bandera en el estado para grabar el evento de bloque
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- Callback: Set state 'guardrail_block_keyword_triggered': True ---")

# # Construya y devuelve una respuesta LLM para detener el flujo y enviar esto en su lugar
        return LlmResponse(
            content=types.Content(
                role="model", # Mimic a response from the agent's perspective
                parts=[types.Part(text=f"I cannot process this request because it contains the blocked keyword '{keyword_to_block}'.")],
            )
# # Nota: también puede establecer un campo ERROR_MESSAGE aquí si sea necesario
        )
    else:
# # Palabra clave no encontrada, permita que la solicitud proceda al LLM
        print(f"--- Callback: Keyword not found. Allowing LLM call for {agent_name}. ---")
        return None # Returning None signals ADK to continue normally

print("✅ block_keyword_guardrail function defined.")


✅ block_keyword_guardrail function defined.


---

** 2 \.Actualice Root Agent para usar la devolución de llamada **

Redefinimos el agente raíz, agregando el parámetro `antes_model_callback` y apuntándolo a nuestra nueva función de barandilla.Le daremos un nuevo nombre de versión para mayor claridad.

* IMPORTANTE:* Necesitamos redefinir los sub-agentes (`Saluding_Agent`,` Farewell_agent`) y la herramienta de estado (`get_weather_stateful`) dentro de este contexto si aún no están disponibles en los pasos anteriores, asegurando que la definición del agente raíz tenga acceso a todos sus componentes.

In [35]:
# Title 2. Actualice Root Agent con antes_Model_Callback


# --- Redefine los sub-agentes (asegura que existan en este contexto) ---
greeting_agent = None
try:
# # Use una constante de modelo definido
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")

farewell_agent = None
try:
# # Use una constante de modelo definido
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")


# --- Defina al agente raíz con la devolución de llamada ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# Verifique todos los componentes antes de continuar
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

# # Use un modelo definido constante como model_gemini_2_5_pro
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # New version name for clarity
        model=root_agent_model,
        description="Main agent: Handles weather, delegates greetings/farewells, includes input keyword guardrail.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate simple greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather requests, greetings, and farewells.",
        tools=[get_weather],
        sub_agents=[greeting_agent, farewell_agent], # Reference the redefined sub-agents
        output_key="last_weather_report", # Keep output_key from Step 4
        before_model_callback=block_keyword_guardrail # <<< Assign the guardrail callback
    )
    print(f"✅ Root Agent '{root_agent_model_guardrail.name}' created with before_model_callback.")

# # --- Crear corredor para este agente, utilizando el mismo servicio de sesión con estado ---
# # Asegurar session_service_stateful existe desde el paso 4
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # Use consistent APP_NAME
            session_service=session_service_stateful # <<< Use the service from Step 4
        )
        print(f"✅ Runner created for guardrail agent '{runner_root_model_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4 is missing.")

else:
    print("❌ Cannot create root agent with model guardrail. One or more prerequisites are missing or failed initialization:")
    if not greeting_agent: print("   - Greeting Agent")
    if not farewell_agent: print("   - Farewell Agent")
    if 'get_weather_stateful' not in globals(): print("   - 'get_weather_stateful' tool")
    if 'block_keyword_guardrail' not in globals(): print("   - 'block_keyword_guardrail' callback")

✅ Sub-Agent 'greeting_agent' redefined.
✅ Sub-Agent 'farewell_agent' redefined.
✅ Root Agent 'weather_agent_v5_model_guardrail' created with before_model_callback.
✅ Runner created for guardrail agent 'weather_agent_v5_model_guardrail', using stateful session service.


---

** 3 \.Interactuar para probar la barandilla **

Probemos el comportamiento de la barandilla.Usaremos la * misma sesión * (`session_id_stateful`) como en el paso 4 para mostrar que el estado persiste en estos cambios.

1. Envíe una solicitud de clima normal (debe pasar la barandilla y ejecutar).
2. Envíe una solicitud que contenga "bloque" (debe ser interceptado por la devolución de llamada).
3. Envíe un saludo (debe pasar la barandilla del agente raíz, ser delegado y ejecutar normalmente).

In [38]:
# Title 3. Interactúa para probar la barandilla de entrada del modelo

# Asegúrese de que el corredor para el agente de barandilla esté disponible
if runner_root_model_guardrail:
  async def run_guardrail_test_conversation():
      print("\n--- Testing Model Input Guardrail ---")

# # Use el corredor para el agente con la devolución de llamada y la ID de sesión de estado existente
  interaction_func = lambda query: call_agent_async(query,
    runner_root_model_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL # <-- Pass correct IDs
  )
# # 1. Solicitud normal (la devolución de llamada lo permite, debe usar Fahrenheit desde el paso 4 Cambio de estado)
  await interaction_func("What is the weather in London?")

# # 2. Solicitud que contiene la palabra clave bloqueada
  await interaction_func("BLOCK the request for weather in Tokyo")

# # 3. Saludo normal (la devolución de llamada permite que el agente raíz, la delegación ocurra)
  await interaction_func("Hello again")


# # Ejecutar la conversación
  await run_guardrail_test_conversation()

# # Opcional: Verifique el estado de la bandera de activación establecida por la devolución de llamada
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- Final Session State (After Guardrail Test) ---")
      print(f"Guardrail Triggered Flag: {final_session.state.get('guardrail_block_keyword_triggered')}")
      print(f"Last Weather Report: {final_session.state.get('last_weather_report')}") # Should be London weather
      print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit')}") # Should be Fahrenheit
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping model guardrail test. Runner ('runner_root_model_guardrail') is not available.")




>>> User Query: What is the weather in London?
--- Callback: block_keyword_guardrail running for agent: weather_agent_v5_model_guardrail ---
--- Callback: Inspecting last user message: 'What is the weather in London?...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v5_model_guardrail. ---


--- Tool: get_weather called for city: London ---
--- Callback: block_keyword_guardrail running for agent: weather_agent_v5_model_guardrail ---
--- Callback: Inspecting last user message: 'What is the weather in London?...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v5_model_guardrail. ---
<<< Agent Response: It's cloudy in London with a temperature of 15°C.


>>> User Query: BLOCK the request for weather in Tokyo
--- Callback: block_keyword_guardrail running for agent: weather_agent_v5_model_guardrail ---
--- Callback: Inspecting last user message: 'BLOCK the request for weather in Tokyo...' ---
--- Callback: Found 'BLOCK'. Blocking LLM call! ---
--- Callback: Set state 'guardrail_block_keyword_triggered': True ---
<<< Agent Response: I cannot process this request because it contains the blocked keyword 'BLOCK'.

>>> User Query: Hello again
--- Callback: block_keyword_guardrail running for agent: weather_agent_v5_model_guardrail ---
--- Callback: Inspecti

--- Tool: say_hello called with name: there ---
<<< Agent Response: Hello, there!


--- Testing Model Input Guardrail ---

--- Final Session State (After Guardrail Test) ---
Guardrail Triggered Flag: True
Last Weather Report: Hello, there!

Temperature Unit: Fahrenheit


---

Observe el flujo de ejecución:

1. ** London Weather: ** La devolución de llamada se ejecuta para `Weather_Agent_V5_Model_Guardrail`, inspecciona el mensaje, imprime la palabra clave" no se encuentra. Permitiendo la llamada LLM ", y devuelve` Ninguno`.El agente procede, llama a la herramienta `get_weather_stateful` (que utiliza la preferencia" Fahrenheit "del cambio de estado del paso 4) y devuelve el clima.Esta respuesta actualiza `last_weather_report` a través de` output_key`.
2. ** Solicitud de bloque: ** La devolución de llamada se ejecuta nuevamente para `weather_agent_v5_model_guardrail`, inspecciona el mensaje, encuentra" bloque ", imprime" Bloqueo de LLM Call \! ", Establece la bandera de estado y devuelve el` llMresponse` predefinido.El LLM subyacente del agente es * nunca se llama * para este turno.El usuario ve el mensaje de bloqueo de la devolución de llamada.
3. ** Hola de nuevo: ** La devolución de llamada se ejecuta para `Weather_agent_V5_Model_Guardrail`, permite la solicitud.El agente raíz luego delega a `saludo_agent`.* Nota: El `antes_model_callback` definido en el agente raíz no se aplica automáticamente a los subciburentes.* El` Saluding_Agent` continúa normalmente, llama a su herramienta `say_hello` y devuelve el saludo.

¡Ha implementado con éxito una capa de seguridad de entrada \!El `antes_model_callback` proporciona un mecanismo poderoso para hacer cumplir las reglas y controlar el comportamiento del agente * antes de * se realizan llamadas LLM costosas o potencialmente riesgosas.A continuación, aplicaremos un concepto similar para agregar barandillas alrededor del uso de herramientas.

## Paso 6: Agregar seguridad \- Argumento de herramienta GuardRail (`antes_tool_callback`)

En el paso 5, agregamos una barandilla para inspeccionar y potencialmente bloquear la entrada del usuario * antes de * llegar al LLM.Ahora, agregaremos otra capa de control * después de * El LLM ha decidido usar una herramienta pero * antes * esa herramienta realmente se ejecuta.Esto es útil para validar los * argumentos * El LLM quiere pasar a la herramienta.

ADK proporciona el `antes_tool_callback` para este propósito preciso.

** ¿Qué es `antes_tool_callback`? **

* Es una función de Python ejecutada solo * antes de * Se ejecuta una función de herramienta específica, después de que el LLM ha solicitado su uso y decidió los argumentos.
*** Propósito: ** Validar los argumentos de la herramienta, evitar la ejecución de la herramienta basada en entradas específicas, modificar los argumentos dinámicamente o hacer cumplir las políticas de uso de recursos.

** Casos de uso comunes: **

*** Validación de argumentos: ** Compruebe si los argumentos proporcionados por el LLM son válidos, dentro de los rangos permitidos o se ajustan a los formatos esperados.
*** Protección de recursos: ** Evite que las herramientas se llamen con entradas que pueden ser costosas, acceder a datos restringidos o causar efectos secundarios no deseados (por ejemplo, bloquear las llamadas de API para ciertos parámetros).
*** Modificación de argumentos dinámicos: ** Ajuste los argumentos basados ​​en el estado de la sesión u otra información contextual antes de que la herramienta se ejecute.

** Cómo funciona: **

1. Defina una función que acepte `herramienta: basetool`,` args: dict [str, any] `y` tool_context: toolcontext`.
* `Tool`: el objeto de herramienta a punto de llamarse (Inspeccionar` Tool.name`).
* `Args`: El Diccionario de Argumentos que el LLM generó para la herramienta.
* `Tool_context`: proporciona acceso al estado de sesión (` tool_context.state`), información del agente, etc.
2. Dentro de la función:
*** Inspeccionar: ** Examine el diccionario `Tool.name` y` Args`.
*** Modificar: ** Cambiar valores dentro del diccionario `Args`*directamente*.Si devuelve `None`, la herramienta se ejecuta con estos args modificados.
*** Bloque/anulación (Guarrail): ** Devuelve un Diccionario ** **.ADK trata este diccionario como el * resultado * de la llamada de la herramienta, completamente * omitiendo * la ejecución de la función de herramienta original.El diccionario debería igualar el formato de retorno esperado de la herramienta que está bloqueando.
*** Permitir: ** Return `None`.ADK procede a ejecutar la función de herramienta real con los argumentos (potencialmente modificados).

** En este paso, lo haremos: **

1. Defina una función `antes_tool_callback` (` block_paris_tool_guardrail`) que verifica específicamente si la herramienta `get_weather_stateful` se llama con la ciudad" París ".
2. Si se detecta "París", la devolución de llamada bloqueará la herramienta y devolverá un diccionario de error personalizado.
3. Actualice nuestro agente root (`weather_agent_v6_tool_guardrail`) para incluir * ambos * el` antes_model_callback` y este nuevo `antes_tool_callback`.
4. Cree un nuevo corredor para este agente, utilizando el mismo servicio de sesión con estado.
5. Pruebe el flujo solicitando el clima para las ciudades permitidas y la ciudad bloqueada ("París").

---

** 1 \.Defina la función de devolución de llamada de la barandilla de la herramienta **

Esta función se dirige a la herramienta `get_weather_stateful`.Verifica el argumento 'Ciudad'.Si es "París", devuelve un diccionario de error que se parece a la propia respuesta de error de la herramienta.De lo contrario, permite que la herramienta se ejecute devolviendo `none`.

In [39]:
# Title 1. Defina la barandilla de Guardia antes_Tool_Callback

# Asegúrese de que las importaciones necesarias estén disponibles
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any # For type hints

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Checks if 'get_weather_stateful' is called for 'Paris'.
    If so, blocks the tool execution and returns a specific error dictionary.
    Otherwise, allows the tool call to proceed by returning None.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name # Agent attempting the tool call
    print(f"--- Callback: block_paris_tool_guardrail running for tool '{tool_name}' in agent '{agent_name}' ---")
    print(f"--- Callback: Inspecting args: {args} ---")

# # --- Lógica de barandilla ---
    target_tool_name = "get_weather_stateful" # Match the function name used by FunctionTool
    blocked_city = "paris"

# # Compruebe si es la herramienta correcta y el argumento de la ciudad coincide con la ciudad bloqueada
    if tool_name == target_tool_name:
        city_argument = args.get("city", "") # Safely get the 'city' argument
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- Callback: Detected blocked city '{city_argument}'. Blocking tool execution! ---")
# # Opcionalmente actualizar el estado
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- Callback: Set state 'guardrail_tool_block_triggered': True ---")

# # Devolver un diccionario que coincida con el formato de salida esperado de la herramienta para errores
# # Este diccionario se convierte en el resultado de la herramienta, omitiendo la ejecución de la herramienta real.
            return {
                "status": "error",
                "error_message": f"Policy restriction: Weather checks for '{city_argument.capitalize()}' are currently disabled by a tool guardrail."
            }
        else:
             print(f"--- Callback: City '{city_argument}' is allowed for tool '{tool_name}'. ---")
    else:
        print(f"--- Callback: Tool '{tool_name}' is not the target tool. Allowing. ---")


# # Si las verificaciones anteriores no devuelven un diccionario, permita que la herramienta ejecute
    print(f"--- Callback: Allowing tool '{tool_name}' to proceed. ---")
    return None # Returning None allows the actual tool function to run

print("✅ block_paris_tool_guardrail function defined.")



✅ block_paris_tool_guardrail function defined.


---

** 2 \.Actualizar el agente root para usar ambas devoluciones de llamada **

Redefinimos el agente raíz nuevamente (`weather_agent_v6_tool_guardrail`), esta vez agregando el parámetro` antes_tool_callback` junto con el `antes_model_callback` del paso 5 \.

* Ejecución autónoma Nota:* Similar al Paso 5, asegurar que todos los requisitos previos (subcásgenes, herramientas, `antes_model_callback`) se definan o estén disponibles en el contexto de ejecución antes de definir este agente.

In [40]:
# Title 2. Actualice Root Agent con ambas devoluciones de llamada (autónomo)

# --- Asegúrese de que se definan los requisitos previos ---
# (Incluya o garantice la ejecución de definiciones para: agente, litellm, corredor, herramientas,
# Modelo constantes, say_hello, say_goodbye, saludo_agent, adiós_agent,
# get_weather_stateful, block_keyword_guardrail, block_paris_tool_guardrail)

# --- Redefine los sub-agentes (asegura que existan en este contexto) ---
greeting_agent = None
try:
# # Utilice un modelo definido constante como model_gpt_4o
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")

farewell_agent = None
try:
# # Utilice un modelo definido constante como model_gpt_4o
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({MODEL_GPT_4O}). Error: {e}")

# --- Defina al agente raíz con ambas devoluciones de llamada ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # New version name
        model=root_agent_model,
        description="Main agent: Handles weather, delegates, includes input AND tool guardrails.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # Keep model guardrail
        before_tool_callback=block_paris_tool_guardrail # <<< Add tool guardrail
    )
    print(f"✅ Root Agent '{root_agent_tool_guardrail.name}' created with BOTH callbacks.")

# # --- Crear corredor, usando el mismo servicio de sesión con estado ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< Use the service from Step 4/5
        )
        print(f"✅ Runner created for tool guardrail agent '{runner_root_tool_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4/5 is missing.")

else:
    print("❌ Cannot create root agent with tool guardrail. Prerequisites missing.")



✅ Sub-Agent 'greeting_agent' redefined.
✅ Sub-Agent 'farewell_agent' redefined.
✅ Root Agent 'weather_agent_v6_tool_guardrail' created with BOTH callbacks.
✅ Runner created for tool guardrail agent 'weather_agent_v6_tool_guardrail', using stateful session service.


---

** 3 \.Interactuar para probar la barandilla de la herramienta **

Probemos el flujo de interacción, nuevamente utilizando la misma sesión de estado (`session_id_stateful`) desde los pasos anteriores.

1. Solicite el clima de "Nueva York": pasa ambas devoluciones de llamada, se ejecuta la herramienta (usando la preferencia de Fahrenheit del estado).
2. Solicite el clima de "París": pasa `antes_model_callback`.LLM decide llamar a `get_weather_stateful (ciudad = 'paris')`.`antes_tool_callback` intercepta, bloquea la herramienta y devuelve el diccionario de error.Agente transmite este error.
3. Solicite clima para "Londres": pasa ambas devoluciones de llamada, la herramienta se ejecuta normalmente.

In [41]:
# Title 3. Interactuar para probar la barandilla del argumento de la herramienta

# Asegúrese de que el corredor para el agente de la barandilla de la herramienta esté disponible
if runner_root_tool_guardrail:
  async def run_tool_guardrail_test():
      print("\n--- Testing Tool Argument Guardrail ('Paris' blocked) ---")

# # Use el corredor para el agente con las devoluciones de llamada y la sesión de estado existente
      interaction_func = lambda query: call_agent_async(query,
      runner_root_tool_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL
  )
# # 1. Ciudad permitida (debería pasar ambas devoluciones de llamada, usar el estado de Fahrenheit)
      await interaction_func("What's the weather in New York?")

# # 2. Ciudad bloqueada (debe pasar la devolución de llamada del modelo, pero estar bloqueado por la devolución de llamada de la herramienta)
      await interaction_func("How about Paris?")

# # 3. Otra ciudad permitida (debería volver a funcionar normalmente)
      await interaction_func("Tell me the weather in London.")

# # Ejecutar la conversación
  await run_tool_guardrail_test()

# # Opcional: Verifique el estado de la bandera de activación del bloque de herramientas
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id= SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- Final Session State (After Tool Guardrail Test) ---")
      print(f"Tool Guardrail Triggered Flag: {final_session.state.get('guardrail_tool_block_triggered')}")
      print(f"Last Weather Report: {final_session.state.get('last_weather_report')}") # Should be London weather
      print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit')}") # Should be Fahrenheit
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping tool guardrail test. Runner ('runner_root_tool_guardrail') is not available.")


--- Testing Tool Argument Guardrail ('Paris' blocked) ---

>>> User Query: What's the weather in New York?


--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_guardrail ---
--- Callback: Inspecting last user message: 'For context:...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v6_tool_guardrail. ---


--- Callback: block_paris_tool_guardrail running for tool 'get_weather_stateful' in agent 'weather_agent_v6_tool_guardrail' ---
--- Callback: Inspecting args: {'city': 'New York'} ---
--- Callback: City 'New York' is allowed for tool 'get_weather_stateful'. ---
--- Callback: Allowing tool 'get_weather_stateful' to proceed. ---
--- Tool: get_weather_stateful called for New York ---
--- Tool: Reading state 'user_preference_temperature_unit': Fahrenheit ---
--- Tool: Generated report in Fahrenheit. Result: {'status': 'success', 'report': 'The weather in New york is sunny with a temperature of 77°F.'} ---
--- Tool: Updated state 'last_city_checked_stateful': New York ---
--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_guardrail ---
--- Callback: Inspecting last user message: 'For context:...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v6_tool_guardrail. ---
<<< Agent Response: The weather in New york is sunny with a temperature o

--- Callback: block_paris_tool_guardrail running for tool 'get_weather_stateful' in agent 'weather_agent_v6_tool_guardrail' ---
--- Callback: Inspecting args: {'city': 'Paris'} ---
--- Callback: Detected blocked city 'Paris'. Blocking tool execution! ---
--- Callback: Set state 'guardrail_tool_block_triggered': True ---
--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_guardrail ---
--- Callback: Inspecting last user message: 'How about Paris?...' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v6_tool_guardrail. ---
<<< Agent Response: I am sorry, I cannot process the request for Paris due to a policy restriction.


>>> User Query: Tell me the weather in London.
--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_guardrail ---
--- Callback: Inspecting last user message: 'Tell me the weather in London....' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v6_tool_guardrail. ---


--- Callback: block_paris_tool_guardrail running for tool 'get_weather_stateful' in agent 'weather_agent_v6_tool_guardrail' ---
--- Callback: Inspecting args: {'city': 'London'} ---
--- Callback: City 'London' is allowed for tool 'get_weather_stateful'. ---
--- Callback: Allowing tool 'get_weather_stateful' to proceed. ---
--- Tool: get_weather_stateful called for London ---
--- Tool: Reading state 'user_preference_temperature_unit': Fahrenheit ---
--- Tool: Generated report in Fahrenheit. Result: {'status': 'success', 'report': 'The weather in London is cloudy with a temperature of 59°F.'} ---
--- Tool: Updated state 'last_city_checked_stateful': London ---
--- Callback: block_keyword_guardrail running for agent: weather_agent_v6_tool_guardrail ---
--- Callback: Inspecting last user message: 'Tell me the weather in London....' ---
--- Callback: Keyword not found. Allowing LLM call for weather_agent_v6_tool_guardrail. ---


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash-exp'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '46s'}]}}

---

Analizar la salida:

1. ** Nueva York: ** El `antes_model_callback` permite la solicitud.El LLM solicita `get_weather_stateful`.El `antes_tool_callback` se ejecuta, inspecciona los args (` {'City': 'Nueva York'} `), ve que no es" París ", imprime" Permitir la herramienta ... "y devuelve` Ninguno`.La función real `get_weather_stateful` se ejecuta, lee" fahrenheit "del estado y devuelve el informe meteorológico.El agente transmite esto, y se guarda a través de `output_key`.
2. ** París: ** El `antes_model_callback` permite la solicitud.El LLM solicita `get_weather_stateful (ciudad = 'paris')`.El `antes_tool_callback` se ejecuta, inspecciona los args, detecta" París ", imprime" Ejecución de la herramienta de bloqueo \! ", Establece el indicador de estado y devuelve el Diccionario de error` {'status': 'Error', 'Error_message': 'Reducción de la política ...'} `.La función real `get_weather_stateful` es ** nunca ejecutada **.El agente recibe el diccionario de error * como si fuera la salida de la herramienta * y formula una respuesta basada en ese mensaje de error.
3. ** Londres: ** se comporta como Nueva York, pasando las devoluciones de llamada y ejecutando la herramienta con éxito.El New London Weather Report sobrescribe el 'Last_Weather_report` en el estado.

Ahora ha agregado una capa de seguridad crucial que controla no solo * lo que * alcanza el LLM, sino también * cómo * las herramientas del agente pueden usarse en función de los argumentos específicos generados por el LLM.Las devoluciones de llamada como `antes_model_callback` y` antes_tool_callback` son esenciales para construir aplicaciones de agente robustas, seguras y compatibles con políticas.

---


## Conclusión: ¡Su equipo de agente está listo!

¡Felicidades!Ha viajado con éxito desde la construcción de un agente meteorológico único y básico hasta la construcción de un equipo sofisticado de múltiples agentes utilizando el Kit de desarrollo de agentes (ADK).

** Recapitulemos lo que has logrado: **

*Comenzó con un agente fundamental ** ** equipado con una sola herramienta (`get_weather`).
*Exploró la flexibilidad multimodelo ** de ADK ** usando litellm, ejecutando la misma lógica central con diferentes LLM como Gemini, GPT-4O y Claude.
*Usted abrazó ** modularidad ** creando subcrescantes especializados (`saludo_agent`,` Farewell_agent`) y habilitando ** la delegación automática ** de un agente raíz.
*Le dio a sus agentes ** Memoria ** usando el estado de la sesión ** **, lo que les permite recordar las preferencias del usuario (`temperatura_unit`) y las interacciones pasadas (` `°_key`).
*Implementó Cross ** Safety BuardRails ** usando tanto `antes_model_callback (bloqueando palabras clave de entrada específicas) como` antes_tool_callback` (ejecución de la herramienta de bloqueo basada en argumentos como la ciudad "París").

A través de la construcción de este equipo progresivo de botes meteorológicos, ha adquirido experiencia práctica con conceptos de ADK centrales esenciales para desarrollar aplicaciones complejas e inteligentes.

** Control de llave: **

*** Agentes y herramientas: ** Los bloques de construcción fundamentales para definir capacidades y razonamiento.Las instrucciones y las documentos claras son primordiales.
*** Corredores y servicios de sesión: ** El sistema de gestión del motor y la memoria que orquesta la ejecución del agente y mantiene el contexto de conversación.
*** Delegación: ** El diseño de equipos de múltiples agentes permite especialización, modularidad y una mejor gestión de tareas complejas.El agente `Descripción` es clave para el flujo automático.
*** Estado de sesión (`ToolContext`,` Output_Key`): ** Esencial para crear agentes conversacionales con el contexto, personalizados y de múltiples vueltas.
*** Callbacks (`antes_model`,` antes_tool`): ** ganchos potentes para implementar seguridad, validación, aplicación de políticas y modificaciones dinámicas*antes de*operaciones críticas (llamadas LLM o ejecución de herramientas).
*** Flexibilidad (`litellm`): ** ADK le permite elegir el mejor LLM para el trabajo, el rendimiento de equilibrio, el costo y las funciones.

** ¿A dónde ir después? **

Su equipo de botes meteorológicos es un gran punto de partida.Aquí hay algunas ideas para explorar más a fondo y mejorar su aplicación:

1. ** API del clima real: ** Reemplace la herramienta `Mock_weather_DB` en su herramienta 'Get_Weather` con una llamada a una API del clima real (como OpenWeathermap, Weatherapi).
2. ** Estado más complejo: ** Almacene más preferencias del usuario (por ejemplo, ubicación preferida, configuración de notificación) o resúmenes de conversación en el estado de la sesión.
3. ** Refina Delegación: ** Experimentar con diferentes instrucciones de agente raíz o descripciones de subagentes para ajustar la lógica de la delegación.¿Podría agregar un agente de "pronóstico"?
4. ** Backbacks avanzadas: **
* Use `After_model_callback` para reformatear o desinfectar la respuesta de la LLM * después de * se genera.
* Use `After_Tool_Callback` para procesar o registrar los resultados devueltos por una herramienta.
* Implementar `antes_agent_callback` o` después_agent_callback` para la lógica de entrada/salida de nivel de agente.
5. ** Manejo de errores: ** Mejore cómo el agente maneja errores de herramienta o respuestas de API inesperadas.Tal vez agregue la lógica de reintento dentro de una herramienta.
6. ** Almacenamiento de sesión persistente: ** Explore alternativas a `InMemorySessionService` para almacenar el estado de la sesión de manera persistente (por ejemplo, el uso de bases de datos como Firestore o Cloud SQL, requiere implementación personalizada o integraciones de ADK futuras).
7. ** CLUYING US: ** Integre su equipo de agentes con un marco web (como Fastapi, como se muestra en la transmisión de ADK, CapidStart) para crear una interfaz de chat en tiempo real.

El kit de desarrollo de agentes proporciona una base robusta para construir aplicaciones sofisticadas con alimentación LLM.Al dominar los conceptos cubiertos en este tutorial (herramientas, estado, delegación y devoluciones de llamada, está bien equipado para abordar sistemas de agente cada vez más complejos.

¡Feliz edificio!